In [ ]:
!pip install transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas

headlines_df = pandas.read_excel('/content/drive/MyDrive/Estudos/IC/Codes/Sentiment/Dataset/Dados Pronto/BTC_News_2015_2023_100.xlsx')
headlines_df.head(5)

,Unnamed: 0,Link,Domain,Title,Description,Date,Data
0,31,https://www.marketpulse.com/newsfeed/eurusd-eu...,MarketPulse,EUR/USD - Euro Ends Dismal 2014 at 1.21 ...,"The news was not as good from Chicago PMI, whi...",1 de jan. de 2015,2015-01-01
1,35,https://www.cityam.com/tesla-founder-elon-musk...,City A.M.,Tesla founder Elon Musk divorces his British w...,"Bitcoin maintains key level, while Tesla's BTC...",2 de jan. de 2015,2015-01-02
2,36,https://premierconstructionnews.com/2015/01/02...,Premier Construction News,The Coach House -,Original BTC were involved in this project sup...,2 de jan. de 2015,2015-01-02
3,96,https://kapital.kz/sport/36388/stiven-dzherrar...,Kapital.kz,Стивен Джеррард покинет «Ливерпуль» в конце се...,BTC. 28595.00$. +0.051 ... «Ливерпуль» купил с...,2 de jan. de 2015,2015-01-02
4,80,https://kapital.kz/finance/36383/mesyachnyy-ra...,Kapital.kz,Месячный расчетный показатель составляет 1 тыс...,В Астане оштрафовали 134 строительные компании...,2 de jan. de 2015,2015-01-02


In [ ]:
import numpy as np

headlines_array = np.array(headlines_df)
headlines_title = list(headlines_array[:,3])
headlines_content = list(headlines_array[:,4])
stocks_list = list(headlines_array[:, 2])
Data = list(headlines_array[:, 6])


print(headlines_title)
print(stocks_list)
print(headlines_content)

In [ ]:
# Extrair títulos e conteúdos
headlines_title = [row[3] for row in headlines_array]
headlines_content = [row[4] for row in headlines_array]

# Juntar os títulos e conteúdos
merged_headlines = [f"{title}: {content}" for title, content in zip(headlines_title, headlines_content)]

# Imprimir os resultados
for headline in merged_headlines:
    print(headline)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [ ]:
results = []
import torch

def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

STRIDE = 25
model.eval()

n=0

for lines, stocks, datas in zip(chunk_list(merged_headlines, STRIDE), chunk_list(stocks_list, STRIDE),chunk_list(Data, STRIDE)):

  input = tokenizer(lines, padding = True, truncation = True,  return_tensors='pt')
  outputs = model(**input)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  print(f"{n+1}/{int(len(merged_headlines)/STRIDE)}")

  for headline, stock, data, pos, neg, neutr in zip(lines,stocks,datas, prediction[:, 0].tolist(), prediction[:, 1].tolist(), prediction[:, 2].tolist() ):
    sentiment_s = (pos-neg)
    results.append({
            "Headline": headline,
            "Stock": stock,
            "Data": data,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neutr,
            "Sentiment Score": sentiment_s
        })

  n+=1


In [ ]:
df150 = pd.DataFrame(results)
df150.to_excel('BTC_News_150.xlsx', index=False)  # O parâmetro index=False evita que o índice seja incluído no arquivo Excel
